In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import seaborn as sns
%matplotlib inline

# Векторизация текста

Для многих задач связанные с текстом нам бы хотелось представлять текст в виде вектора, чтобы похожие слова или предложения располагались рядом с друг другом. Для чего это нужно?. Например для задач поиска, нам хотелось бы выдавать ответ так, чтобы наиболее похожие статьи находились в верху списка. Такой тип представления текстов и слов называется дистрибутивная семантика. 
Каким же образом мы можем представить слова в таком виде?

## PMI

Логичным методом представления слова является, контекст в котором оно употребляются в тексте
Контекстом мы будем считать слова, которые находятся рядом с искомым словом в окне определенного размера, например для слова <i>лес</i> и окна 3, контекст выглядит следующим образом
Житель <b>местности, покрытой хвойным</b> <i>лесом</i>, <b>связывает с «деревом»</b> прежде всего образ ели или сосны.
Исходя из этой идеи, мы можем посчитать вектор слова следующим образом

- Посчитать слова которые встречаются с данным словом в одном контексте
- Создать вектор где в определенное значение записана частота определенного слова из словаря

В таком случае похожесть между словами мы можем измерять с помощью косинусного расстояния между их векторами
Однако такой способ имеет большой недостаток, а именно часто встречаемые слова (например предлоги) будут иметь очень большой вес. Чтобы уменьшить влияние таких слов, вместо частоты встречаемости слов, будем считать PMI (pointwise mutual information)
$$\large{PMI = \log{\frac{p(u, v)}{p(v)p(u)}}=\log{\frac{n_{uv}n}{n_un_v}}}$$
Однако из с данным способом представления встречаемости слов есть проблема, для редко встречаемых между собой слов получается большое отрицательно значение, а если слова никогда не встречаются между собой то значение равно $-\infty$. Поэтому на практике пользуются pPMI
$$\large{pPMI = \max{(0, PMI)}}$$

## Truncated SVD

И так у нас имеется способ представление совстречаемости слов с помощью pPMI. Так же мы можем мерить похожесть слов между собой с помощью косинусного расстояния. Однако у нас есть другая проблема а именно, размерность вектора каждого слова равна длинне словаря и затраты на вычисление похожести слов слишком большие. Поэтому воспользуюмся техникой снижения размерности а именно Truncated SVD
Truncated SVD представляет нашу матрицу размера (W*W) в виде произведения двух матриц (WK) и (KW), где K - новая размерность для наших векторов
<img src="./img/pmi_svd.png">
Для того что найти наилучшие матрицы для такого произведения мы решаем задачу минимизации расстояния между исходной матрицей и скалярного произведения новых матриц
$\large{||X-X_f||^2 \rightarrow min}$

## GloVe

GloVe - Global Vectors for Word Representation алгоритм представления слов в виде векторов, был разработан в университет Стендфорд. Почитать о нем можно по <a href="https://nlp.stanford.edu/projects/glove/">ссылке</a>, <a href="https://github.com/stanfordnlp/GloVe">git</a> проекта где можно скачать вектора
GloVe похож на вектора которые получаются с помощью SVD из pPMI матрицы, однако во время поиска матрицы представления слов, минимизируется другая ошибка и матрица заполнена логорифомом встречаемости слов:
$$\large{\sum_{u \in W}\sum_{v \in W}f(n_{uv})(\phi_u\theta_v + b_u + b_v - \log{n_{uv}})^2}$$
Это делается для того чтобы уменьшить влияние часто встречаемых слов

## Skip-gram

В данном случае мы пытаемся построить модель, которая бы предсказывала контекст по заданному слову:
$$\large{p(\omega_{i-k}..\omega_{i+k}|\omega_{i}) = \prod_{-k\le j\le k, j\ne 0}p(\omega_{i+j}|\omega{i})}$$
Где вероятность встречаемости каждого слова, представлена как:
$$\large{p(u|v) = \frac{\exp{\phi_u\theta_v}}{\sum_{\dot{u}\in W}\exp{\phi_\dot{u}\theta_v}}}$$
Так же как в версии с Truncated SVD у нас есть матрицы $\Phi$ и $\Theta$, в которых находятся векторы слов, в skip-gram мы максимизируем логарифмическое правдоподобие, вместо минимизации MSE
$$\large{L = \sum_{u \in W}\sum_{v \in W}n_{uv}\log{p(u|v)}}$$
Однако на практике данной моделью пользуются редко, поскольку ее обучение занимает очень много времени из-за того что нам нужно считать софтмакс по всему словарю

## Skip-gram Negative Sampling

Данная модель абсолютно аналогична Skip-gram за исключением того что функция максимизации состоит из 2 частей:

- для положительных примеров (слов из контекста) 
$$\large{L_{positive} = \sum_{u \in W}\sum_{v \in W}n_{uv}\log{\sigma(\phi_u\theta_v)}}$$

- для отрицательных примеров (слов вне контекста), выбираем случайные k слов 
$$\large{L_{negative} = kE_v\log{\sigma(-\phi_u\theta_v)}}$$

Полная метрика выглядит как сумма позитивной и негативной метрик
$$\large L = L_{positive}+L_{negative}$$
Интересным фактом является то что оптимальное значение для скалярного произведения skip-gram векторов является смещенная оценка PMI
$$\large{sPMI = PMI - \log{k}}$$

## CBOW

Архитектура CBOW аналогична Skip-gram, с одним отличием, мы пытаемся максимизировать вероятность найти слово по заданному контексту

## Doc2Vec

Doc2Vec это набор моделей которые к векторам слов, добавляют вектор документов для этих слов, сделанно это для того чтобы:

- Получить векторное представление документа
- Использовать информацию о документе для того чтобы различать омонимы

В Doc2Vec представленно 2 модели

- Distributed memory version of Paragraph Vector

$$\large{p(\omega_{i}|\omega_{i-k}..\omega_{i+k}, d)}$$

- Distributed Bag of Words version of Paragraph Vector

$$\large{p(\omega_{i-k}..\omega_{i+k}|d)}$$

Skip-Gram, CBOW, Doc2Vec представлены в пакете <a href="https://radimrehurek.com/gensim/">gensim</a>

## FastText

Данная модель похожа на Skip-gram Negative Sampling, только вместо скалярного произведения вектора слов, мы используем скалярное произведение для сумм char-gram
Пример 3char-gram для слова <i>семантика</i>: __с, _се, сем, ема, ман, ант, нти, тик, ика
$$\large{\log{\sigma(\phi_u\theta_v)} = \log{\sigma(\sum_{c \in word}\phi_u c_v})}$$
<a href="https://github.com/facebookresearch/fastText/">git FastText</a>

## StarSpace

В реальных задачах нам чаще нужны вектора целых предложений, вместо векторов слов нам нужны вектора целых предложений, существует несколько подходов для их получения

- среднее среди всех векторов в тексте
- взвешенная сумма по idf векторов в тексте
- пакет StarSpace

Пакет StarSpace можно найти на <a href="https://github.com/facebookresearch/StarSpace">git</a>, идея данной модели очень схожа с идеей FastText только вместо похожести слов, мы ищем похожие предложения, каждое из которых представленно в виде сумм векторов слов. В результате мы получаем оптимизированные вектора слов для представления предложений

In [2]:
import numpy as np
import gensim
import gensim.downloader as api
from sklearn.base import BaseEstimator, TransformerMixin

/home/imd/miniconda3/envs/mlisuct/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
glove_model = api.load("glove-twitter-25")

In [4]:
model = api.load("word2vec-google-news-300")

In [5]:
model.most_similar("apple")

[('apples', 0.720359742641449),
 ('pear', 0.6450697183609009),
 ('fruit', 0.6410146355628967),
 ('berry', 0.6302295327186584),
 ('pears', 0.613396167755127),
 ('strawberry', 0.6058261394500732),
 ('peach', 0.6025872230529785),
 ('potato', 0.5960935354232788),
 ('grape', 0.5935863852500916),
 ('blueberry', 0.5866668224334717)]

In [6]:
# king - man + woman
model.most_similar(positive=["woman", "king"], negative=["man"])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674735069275),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454)]

In [7]:
class Word2VecModel(BaseEstimator, TransformerMixin):
    
    def __init__(self, model):
        self.model = model
        
    def get_mean_vector(self, text):
        v = np.zeros(300)
        c = 0
        for word in text.split(" "):
            if word in self.model:
                v += self.model.get_vector(word)
                c += 1
        c = max(1, c)
        return v / c
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([self.get_mean_vector(x) for x in X])

In [8]:
w2v = Word2VecModel(model)